# Bivariate choropleth map using Plotly Choropleth Mapbox

Based on the example by [_empet_](https://chart-studio.plotly.com/~empet/15191/texas-bivariate-choropleth-assoc/#/)

In [ ]:
import numpy as np
import pandas as pd
import json, requests
import plotly.graph_objs as go
import os

## Set defaults for variables

In [ ]:
# Define some variables for later use
plot_title = 'Bivariate choropleth map using Ploty'
width = 1000  # Width of the final map container
ratio = 0.8  # Ratio of height to width
height = width * ratio  # Width of the final map container
center_lat = 31.968599  # Latitude of the center of the map
center_lon = -99.901810  # Longitude of the center of the map
map_zoom = 5  # Zoom factor of the map
hover_x_label = 'Label x variable'  # Label to appear on hover
hover_y_label = 'Label y variable'  # Label to appear on hover
borders_width = 0.5  # Width of the geographic entity borders
borders_color = '#f8f8f8'  # Color of the geographic entity borders

# Define settings for the legend
top=1  # Vertical position of the top right corner (0: bottom, 1: top)
right=1  # Horizontal position of the top right corner (0: left, 1: right)
box_w=0.04  # Width of each rectangle
box_h=0.04  # Height of each rectangle
line_color='#f8f8f8'  # Color of the rectagles' borders
line_width=0  # Width of the rectagles' borders
legend_x_label = 'Higher x value'  # x variable label for the legend 
legend_y_label = 'Higher y value'  # y variable label for the legend

## Define functions

In [ ]:
"""
Function to load GeoJSON file with geographical data of the entities
"""

def load_geojson(geojson_url, data_dir='data', local_file='imported-geojson'):
    
    # Make sure data_dir is a string
    data_dir = str(data_dir)
    
    # Set name for the file to be saved
    geojson_file = data_dir + '/' + str(local_file) + '.geojson'

    # Create folder for data if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # Download GeoJSON in case it doesn't exist
    if not os.path.exists(geojson_file):

        # Make http request for remote file data
        geojson_request = requests.get(geojson_url)

        # Save file data to local copy
        with open(geojson_file, 'wb') as file:
            file.write(geojson_request.content)

    # Load GeoJSON file
    geojson = json.load(open(geojson_file, 'r'))
    
    return geojson

"""
Function that assigns a value (x) to one of three bins (0, 1, 2).
The break points for the bins can be defined by break_a and break_b.
"""

def set_interval_value(x, break_1, break_2):
    if x <= break_1: 
        return 0
    elif break_1 < x <= break_2: 
        return 1
    else: 
        return 2

"""
Function that adds a column 'biv_bins' to the dataframe containing the 
position in the 9-color matrix for the bivariate colors
    
Arguments:
    df: Dataframe
    x: Name of the column containing values of the first variable
    y: Name of the column containing values of the second variable

"""

def prepare_df(df, x='x', y='y'):
    
    # Check if arguments match all requirements
    if len(df[x]) != len(df[y]):
        raise ValueError('ERROR: The list of x and y coordinates must have the same length.')
    
    # Calculate break points at percentiles 33 and 66
    x_breaks = np.percentile(df[x], [33, 66])
    y_breaks = np.percentile(df[y], [33, 66])
    
    # Assign values of both variables to one of three bins (0, 1, 2)
    x_bins = [set_interval_value(value_x, x_breaks[0], x_breaks[1]) for value_x in df[x]]
    y_bins = [set_interval_value(value_y, y_breaks[0], y_breaks[1]) for value_y in df[y]]
    
    # Calculate the position of each x/y value pair in the 9-color matrix of bivariate colors
    df['biv_bins'] = [int(value_x + 3 * value_y) for value_x, value_y in zip(x_bins, y_bins)]
    
    return df
   


"""
Function to create a color square containig the 9 colors to be used as a legend
"""

def create_legend(fig, colors):
    
    # Reverse the order of colors
    legend_colors = colors[:]
    legend_colors.reverse()

    # Calculate coordinates for all nine rectangles
    coord = []

    # Adapt height to ratio to get squares
    width = box_w
    height = box_h/ratio
    
    # Start looping through rows and columns to calculate corners the squares
    for row in range(1, 4):
        for col in range(1, 4):
            coord.append({
                'x0': round(right-(col-1)*width, 4),
                'y0': round(top-(row-1)*height, 4),
                'x1': round(right-col*width, 4),
                'y1': round(top-row*height, 4)
            })

    # Create shapes (rectangles)
    for i in range(len(coord)):
        # Add rectangle
        fig.add_shape(go.layout.Shape(
            type='rect',
            fillcolor=legend_colors[i],
            line=dict(
                color=line_color,
                width=line_width,
            ),
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='top',
            x0=coord[i]['x0'],
            y0=coord[i]['y0'],
            x1=coord[i]['x1'],
            y1=coord[i]['y1'],
        ))
    
        # Add text for first variable
        fig.add_annotation(
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='top',
            x=coord[8]['x1'],
            y=coord[8]['y1'],
            showarrow=False,
            text=legend_x_label + ' 🠒',
            font=dict(
                color='#333',
                size=9,
            ),
            borderpad=0,
        )
        
        # Add text for second variable
        fig.add_annotation(
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',
            x=coord[8]['x1'],
            y=coord[8]['y1'],
            showarrow=False,
            text=legend_y_label + ' 🠒',
            font=dict(
                color='#333',
                size=9,
            ),
            textangle=270,
            borderpad=0,
        )
    
    return fig


"""
Function to create the map

Arguments:
    df: The dataframe that contains all the necessary columns
    colors: List of 9 blended colors
    x: Name of the column that contains values of first variable (defaults to 'x')
    y: Name of the column that contains values of second variable (defaults to 'y')
    ids: Name of the column that contains ids that connect the data to the GeoJSON (defaults to 'id')
    name: Name of the column conatining the geographic entity to be displayed as a description (defaults to 'name')
"""

def create_bivariate_map(df, colors, geojson, x='x', y='y', ids='id', name='name'):
    
    if len(colors) != 9:
        raise ValueError('ERROR: The list of bivariate colors must have a length eaqual to 9.')
    
    # Prepare the dataframe with the necessary information for our bivariate map
    df_plot = prepare_df(df, x=x, y=y)
    
    # Create the figure
    fig = go.Figure(go.Choroplethmapbox(
        geojson=geojson,
        locations=df_plot['id'],
        z=df_plot['biv_bins'],
        marker_line_width=.5,
        colorscale=[
            [0/8, colors[0]],
            [1/8, colors[1]],
            [2/8, colors[2]],
            [3/8, colors[3]],
            [4/8, colors[4]],
            [5/8, colors[5]],
            [6/8, colors[6]],
            [7/8, colors[7]],
            [8/8, colors[8]],
        ],
        customdata=df_plot[['name', 'id', 'x', 'y']],  # Add data to be used in hovertemplate
        hovertemplate='<br>'.join([  # Data to be displayed on hover
            '<b>%{customdata[0]}</b> (ID: %{customdata[1]})',
            hover_x_label + ': %{customdata[2]}',
            hover_y_label + ': %{customdata[3]}',
            '<extra></extra>',  # Remove secondary information
        ])
    ))

    # Add some more details
    fig.update_layout(
        title=plot_title,
        mapbox_style='white-bg',
        width=width,
        height=height,
        autosize=True,
        mapbox=dict(
            center=dict(lat=center_lat, lon=center_lon),  # Set map center
            zoom=map_zoom  # Set zoom
        ),
    )

    fig.update_traces(
        marker_line_width=borders_width,  # Width of the geographic entity borders
        marker_line_color=borders_color,  # Color of the geographic entity borders
        showscale=False,  # Hide the colorscale
    )

    # Add the legend
    fig = create_legend(fig, colors)
    
    fig.show()

## Define lists of blended colors 

1) "pink-blue" by [Joshua Stevens](http://www.joshuastevens.net/cartography/make-a-bivariate-choropleth-map/)
2) "teal-red" by [Joshua Stevens](http://www.joshuastevens.net/cartography/make-a-bivariate-choropleth-map/)
3) "blue-orange" from the [ArcGIS Website](https://pro.arcgis.com/de/pro-app/latest/help/mapping/layer-properties/bivariate-colors.htm)

![Three examples of color sets](img/colors.png)

In [ ]:
# Define sets of 9 colors to be used
# Order: bottom-left, bottom-center, bottom-right, center-left, center-center, center-right, top-left, top-center, top-right
color_sets = {
    'pink-blue':   ['#e8e8e8', '#ace4e4', '#5ac8c8', '#dfb0d6', '#a5add3', '#5698b9', '#be64ac', '#8c62aa', '#3b4994'],
    'teal-red':    ['#e8e8e8', '#e4acac', '#c85a5a', '#b0d5df', '#ad9ea5', '#985356', '#64acbe', '#627f8c', '#574249'],
    'blue-organe': ['#fef1e4', '#fab186', '#f3742d',  '#97d0e7', '#b0988c', '#ab5f37', '#18aee5', '#407b8f', '#5c473d']
}

## Start preparing our data

In [ ]:
"""
Get data and write it to a dataframe containing
    id: Id of the geographic entity (needs to be the same as references in the geospatial data)
    x: Values of the first variable
    y: Values of the second variable
"""

# Get data for first variable (x)
url_x = 'https://raw.githubusercontent.com/empet/Datasets/master/Texas-housing.csv'
df_x = pd.read_csv(url_x, usecols=['GEO.id2', 'GEO.display-label', 'hucen42010', 'huest72016'])

# Rename columns
df_x.columns = ['id', 'name', 'x_2010', 'x_2016']

# Calculate the ratio between two years
df_x['x'] = df_x['x_2016'] / df_x['x_2010']


# Get data for second variable (y)
url_y = 'https://raw.githubusercontent.com/empet/Datasets/master/Texas-population.csv'
df_y = pd.read_csv(url_y, usecols=['GEO.id2', 'rescen42010', 'respop72016'], skiprows=[1])

# Rename columns
df_y.columns = ['id', 'y_2010', 'y_2016']

# Calculate the ratio between two years
df_y['y'] = df_y['y_2016'] / df_y['y_2010']


# Merge both dataframes
df = pd.merge(df_x, df_y[['id', 'y_2010', 'y_2016', 'y']], how="left", left_on=['id'], right_on=['id']);

## Load geospatial data (GeoJSON)

In [ ]:
# Define URL of the GeoJSON file
geojson_url = 'https://data.texas.gov/api/geospatial/48ag-x9aa?method=export&format=GeoJSON'

# Load GeoJSON file into
geojson = load_geojson(geojson_url)

# Iterate over JSON to add necessary 'id' property (Alternatively, use featureidkey when building the fig)
for i in range(len(geojson['features'])):
    # Assign the county id to a new 'id' property for later linking to dataframe
    # In our dataframe all ids start with the state code 48, so we'll add it
    geojson['features'][i]['id'] = '48' + geojson['features'][i]['properties']['fips_code']

## Create the bivariate map from the original example by empet

In [ ]:
# Override some variables
plot_title = 'Texas housing and population'
hover_x_label = 'Housing ratio'  # Label to appear on hover
hover_y_label = 'Population ratio'  # Label to appear on hover
legend_x_label = 'Higher housing ratio'  # x variable label for the legend 
legend_y_label = 'Higher pop ratio'  # y variable label for the legend

In [ ]:
# Create our bivariate map
create_bivariate_map(df, color_sets['pink-blue'], geojson)